<a href="https://colab.research.google.com/github/dafon1986/Elecciones/blob/main/Elecciones%202016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install helpers

In [1]:
!pip install geopandas


     |████████████████████████████████| 972kB 5.8MB/s 
     |████████████████████████████████| 6.5MB 25.1MB/s 
     |████████████████████████████████| 14.8MB 262kB/s 


In [3]:
!pip install contextily

     |████████████████████████████████| 18.3MB 261kB/s 


In [4]:
import requests
import pandas as pd
import geopandas as gdf

import helpers
%matplotlib inline
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import contextily as cx
from urllib.request import urlopen

plt.rcParams['figure.figsize']=(10,10)

In [5]:
!git clone https://github.com/dafon1986/Elecciones.git

Cloning into 'Elecciones'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 46 (delta 22), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (46/46), done.


In [6]:
elecciones2016 = pd.read_csv('/content/Elecciones/usa-2016-presidential-election-by-county.csv',header=None,sep=';',engine='python',encoding='latin1')

In [7]:
elecciones = elecciones2016.rename(columns=elecciones2016.iloc[0]) 

In [8]:
elecciones = elecciones.reindex(elecciones2016.index.drop(0)) 

In [9]:
elecciones.head(5)

,State,ST,Fips,County,Precincts,Votes,Democrats 08 (Votes),Democrats 12 (Votes),Republicans 08 (Votes),Republicans 12 (Votes),Republicans 2016,Democrats 2016,Green 2016,Libertarians 2016,Republicans 2012,Republicans 2008,Democrats 2012,Democrats 2008,Less Than High School Diploma,At Least High School Diploma,At Least Bachelors's Degree,Graduate Degree,School Enrollment,Median Earnings 2010,White (Not Latino) Population,African American Population,Native American Population,Asian American Population,Other Race or Races,Latino Population,Children Under 6 Living in Poverty,Adults 65 and Older Living in Poverty,Total Population,Preschool.Enrollment.Ratio.enrolled.ages.3.and.4,Poverty.Rate.below.federal.poverty.threshold,Gini.Coefficient,Child.Poverty.living.in.families.below.the.poverty.line,Management.professional.and.related.occupations,Service.occupations,Sales.and.office.occupations,...,Votes16 Kahnl,Votes16 La Rivag,Votes16 Hoeflingt,Votes16 Kenistonc,Votes16 Smithm,Votes16 Atwoodf,Votes16 Kennedya,Votes16 Kopitkek,Votes16 Kotlikoffl,Votes16 Lyttleb,Votes16 Maldonadoj,Votes16 Maturenm,Votes16 Scottr,Votes16 Silvar,Votes16 Soltysike,Votes16 Vacekd,Votes16 Copelands,Votes16 Jacobp,Votes16 Whitej,Votes16 Mooreheadm,Votes16 None Of These Candidates,Votes16 Duncanr,Votes16 Skewesp,Votes16 Giordanir,total16,Other16 Frac,Rep16 Frac2,Dem16 Frac2,Name Prev,Statecode Prev,total08,total12,other08,other12,Other12 Frac,Other08 Frac,Rep12 Frac2,Rep08 Frac2,Dem12 Frac2,Dem08 Frac2
1,Georgia,GA,13089,"DeKalb County, Georgia",192,295871,254594,238224,65581,64392,16.06477147135069,80.82272341662413,NaN,3.112505112025173,21.024929391213494,20.3475600675139,77.78361876153005,78.99188343923748,12.1,87.9,38.7,15.1,76.15,31275.05492,30.05,53.6,0.2,4.6,1.65,9.95,25.75,10.75,712476,56.9,15.75,0.47200000000000003,24.0,41.3,15.1,25.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,295871.0,0.031125051120251,0.165808513161842,0.834191486838157,DeKalb,13,322304,306265,2129,3649,0.011914518472564,0.006605564932486,0.212784518994369,0.204828609354259,0.7872154810056301,0.79517139064574
2,Texas,TX,48487,"Wilbarger County, Texas",7,4101,1196,971,3283,2956,77.20068276030236,19.6781272860278,0.316995854669592,2.8041940990002443,74.2713567839196,72.80993568418718,24.396984924623116,26.524728321135505,26.4,73.6,15.9,5.9,69.25,22438.66508,63.3,8.9,0.7000000000000001,0.4,1.1,25.55,52.95,11.6,13630,32.4,24.3,0.454,37.05,26.25,25.5,18.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4101.0,0.028041940990002002,0.7968789327963751,0.20312106720362402,Wilbarger,48,4509,3980,30,53,0.013316582914572002,0.0066533599467730005,0.752737458619811,0.732976110739004,0.24726254138018802,0.267023889260995
3,Virginia,VA,51111,"Lunenburg County, Virginia",12,5577,2703,2684,2900,2969,57.48610363994979,39.913932221624535,0.448269679038909,1.5779092702169621,51.778862922915934,51.32743362831859,46.808510638297875,47.84070796460177,28.5,71.5,10.0,3.0,72.35,24528.98095,59.3,35.7,0.15,0.1,1.45,3.35,32.35,16.95,12911,35.4,18.55,0.392,27.35,21.15,19.65,26.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5577.0,0.015779092702169,0.5902061855670101,0.409793814432989,Lunenburg,51,5650,5734,47,81,0.014126264387861001,0.008318584070796001,0.525207854236688,0.517579867927895,0.474792145763311,0.48242013207210405
4,Georgia,GA,13297,"Walton County, Georgia",21,40425,8469,8148,27253,29036,76.91527520098948,20.479901051329623,NaN,2.60482374768089,77.19876635116452,75.64814300782768,21.663298947144526,23.50802198412258,19.6,80.4,17.3,5.3,73.75,29155.95143,78.9,15.45,0.30000000000000004,1.1,1.0,3.1,22.25,8.85,83005,44.2,12.15,0.40700000000000003,17.7,26.8,14.8,29.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40425.0,0.026048237476808,0.789723661485319,0.21027633851468003,Walton,13,36026,3

In [10]:
elecciones.columns

Index(['State', 'ST', 'Fips', 'County', 'Precincts', 'Votes',
       'Democrats 08 (Votes)', 'Democrats 12 (Votes)',
       'Republicans 08 (Votes)', 'Republicans 12 (Votes)',
       ...
       'total08', 'total12', 'other08', 'other12', 'Other12 Frac',
       'Other08 Frac', 'Rep12 Frac2', 'Rep08 Frac2', 'Dem12 Frac2',
       'Dem08 Frac2'],
      dtype='object', length=159)

In [11]:
elecciones1 = elecciones.drop(columns=['Democrats 08 (Votes)', 'Democrats 12 (Votes)',
       'Republicans 08 (Votes)', 'Republicans 12 (Votes)','total08', 'total12', 'other08', 'other12', 'Other12 Frac',
       'Other08 Frac', 'Rep12 Frac2', 'Rep08 Frac2', 'Dem12 Frac2',
       'Dem08 Frac2','Republicans 2012',
       'Republicans 2008', 'Democrats 2012', 'Democrats 2008','CA', 'S', 'MAR',
       'CFS', 'ACFS', 'Mean Alc', 'Max Alc', 'Mixedness', 'elevation',
       'Annual Prcp','Winter Prcp', 'Summer Prcp', 'Spring Prcp', 'Autumn Prcp',
       'Annual Tavg', 'Annual Tmax', 'Annual Tmin', 'Winter Tavg',
       'Winter Tmax', 'Winter Tmin', 'Summer Tavg', 'Summer Tmax',
       'Summer Tmin', 'Spring Tavg', 'Spring Tmax', 'Spring Tmin',
       'Autumn Tavg', 'Autumn Tmax', 'Autumn Tmin', 'Nearest County','temp', 'precip', 'Temp Bins', 'Lat Bins', 'Lon Bins', 'Precip Bins',
       'Elevation Bins','Other16 Frac', 'Rep16 Frac2', 'Dem16 Frac2', 'Name Prev',
       'Statecode Prev', 'total08'])

In [12]:
elecciones1

,State,ST,Fips,County,Precincts,Votes,Republicans 2016,Democrats 2016,Green 2016,Libertarians 2016,Less Than High School Diploma,At Least High School Diploma,At Least Bachelors's Degree,Graduate Degree,School Enrollment,Median Earnings 2010,White (Not Latino) Population,African American Population,Native American Population,Asian American Population,Other Race or Races,Latino Population,Children Under 6 Living in Poverty,Adults 65 and Older Living in Poverty,Total Population,Preschool.Enrollment.Ratio.enrolled.ages.3.and.4,Poverty.Rate.below.federal.poverty.threshold,Gini.Coefficient,Child.Poverty.living.in.families.below.the.poverty.line,Management.professional.and.related.occupations,Service.occupations,Sales.and.office.occupations,Farming.fishing.and.forestry.occupations,Construction.extraction.maintenance.and.repair.occupations,Production.transportation.and.material.moving.occupations,White,Black,Hispanic,Asian,Amerindian,...,Homicide.rate,Injury.deaths,Infant.mortality,Geo Shape,Name 16,reporting,Votes16 Trumpd,Votes16 Clintonh,Votes16 Johnsong,Votes16 Steinj,Votes16 Castled,Votes16 De La Fuenter,Est Votes Remaining,Votes16 Mcmulline,Votes16 Hedgesj,Votes16 Kahnl,Votes16 La Rivag,Votes16 Hoeflingt,Votes16 Kenistonc,Votes16 Smithm,Votes16 Atwoodf,Votes16 Kennedya,Votes16 Kopitkek,Votes16 Kotlikoffl,Votes16 Lyttleb,Votes16 Maldonadoj,Votes16 Maturenm,Votes16 Scottr,Votes16 Silvar,Votes16 Soltysike,Votes16 Vacekd,Votes16 Copelands,Votes16 Jacobp,Votes16 Whitej,Votes16 Mooreheadm,Votes16 None Of These Candidates,Votes16 Duncanr,Votes16 Skewesp,Votes16 Giordanir,total16
1,Georgia,GA,13089,"DeKalb County, Georgia",192,295871,16.06477147135069,80.82272341662413,NaN,3.112505112025173,12.1,87.9,38.7,15.1,76.15,31275.05492,30.05,53.6,0.2,4.6,1.65,9.95,25.75,10.75,712476,56.9,15.75,0.47200000000000003,24.0,41.3,15.1,25.45,0.05,8.15,9.85,30.05,53.6,9.95,4.6,0.2,...,13.97,51.7,8.5,"{""type"": ""Polygon"", ""coordinates"": [[[-84.3484...",DeKalb,192,47531,239131,9209,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,295871.0
2,Texas,TX,48487,"Wilbarger County, Texas",7,4101,77.20068276030236,19.6781272860278,0.316995854669592,2.8041940990002443,26.4,73.6,15.9,5.9,69.25,22438.66508,63.3,8.9,0.7000000000000001,0.4,1.1,25.55,52.95,11.6,13630,32.4,24.3,0.454,37.05,26.25,25.5,18.1,2.35,12.95,14.75,63.3,8.9,25.55,0.4,0.7000000000000001,...,NaN,70.4,NaN,"{""type"": ""Polygon"", ""coordinates"": [[[-99.4752...",Wilbarger,7,3166,807,115,13,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4101.0
3,Virginia,VA,51111,"Lunenburg County, Virginia",12,5577,57.48610363994979,39.913932221624535,0.448269679038909,1.5779092702169621,28.5,71.5,10.0,3.0,72.35,24528.98095,59.3,35.7,0.15,0.1,1.45,3.35,32.35,16.95,12911,35.4,18.55,0.392,27.35,21.15,19.65,26.75,1.9,10.75,19.85,59.3,35.7,3.35,0.1,0.15,...,NaN,92.5,NaN,"{""type"": ""Polygon"", ""coordinates"": [[[-78.3973...",Lunenburg,12,3206,2226,88,25,NaN,NaN,0,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5577.0
4,Georgia,GA,13297,"Walton County, Georgia",21,40425,76.91527520098948,20.479901051329623,NaN,2.60482374768089,19.6,80.4,17.3,5.3,73.75,29155.95143,78.9,15.45,0.30000000000000004,1.1,1.0,3.1,22.25,8.85,83005,44.2,12.15,0.40700000000000003,17.7,26.8,14.8,29.15,0.4,14.05,14.75,78.9,15.45,3.1,1.1,0.30000000000000004,...,4.35,59.7,7.8,"{""type"": ""Polygon"", ""coordinates"": [[[-83.9156...",Walton,21,31093,8279,1053,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40425.0
5,North Carolina,NC,37011,"Avery County, North Carolina",19,8060,77.24565756823822,20.719602977667495,NaN,2.034739454094293,18.7,81.3,20.3,6.0,76.25,19670.20603,89.85,3.3,0.30000000000000004,0.4,2.1,4.1,22.7,10.0,17850,52.9,15.6,0.529,20.85,29.7,22.15,21.35,1.75,12.25,12.75,89.85,3

In [13]:
elecciones1.columns

Index(['State', 'ST', 'Fips', 'County', 'Precincts', 'Votes',
       'Republicans 2016', 'Democrats 2016', 'Green 2016', 'Libertarians 2016',
       'Less Than High School Diploma', 'At Least High School Diploma',
       'At Least Bachelors's Degree', 'Graduate Degree', 'School Enrollment',
       'Median Earnings 2010', 'White (Not Latino) Population',
       'African American Population', 'Native American Population',
       'Asian American Population', 'Other Race or Races', 'Latino Population',
       'Children Under 6 Living in Poverty',
       'Adults 65 and Older Living in Poverty', 'Total Population',
       'Preschool.Enrollment.Ratio.enrolled.ages.3.and.4',
       'Poverty.Rate.below.federal.poverty.threshold', 'Gini.Coefficient',
       'Child.Poverty.living.in.families.below.the.poverty.line',
       'Management.professional.and.related.occupations',
       'Service.occupations', 'Sales.and.office.occupations',
       'Farming.fishing.and.forestry.occupations',
       'Cons

In [14]:
elecciones1['Votos 16 otros'] = elecciones1[['Votes16 Steinj', 'Votes16 Castled',
       'Votes16 De La Fuenter', 'Est Votes Remaining', 'Votes16 Mcmulline',
       'Votes16 Hedgesj', 'Votes16 Kahnl', 'Votes16 La Rivag',
       'Votes16 Hoeflingt', 'Votes16 Kenistonc', 'Votes16 Smithm',
       'Votes16 Atwoodf', 'Votes16 Kennedya', 'Votes16 Kopitkek',
       'Votes16 Kotlikoffl', 'Votes16 Lyttleb', 'Votes16 Maldonadoj',
       'Votes16 Maturenm', 'Votes16 Scottr', 'Votes16 Silvar',
       'Votes16 Soltysike', 'Votes16 Vacekd', 'Votes16 Copelands',
       'Votes16 Jacobp', 'Votes16 Whitej', 'Votes16 Mooreheadm',
       'Votes16 None Of These Candidates', 'Votes16 Duncanr',
       'Votes16 Skewesp', 'Votes16 Giordanir']].sum(axis=1)

In [15]:
elecciones2 = elecciones1.drop(columns=['Votes16 Steinj', 'Votes16 Castled',
       'Votes16 De La Fuenter', 'Est Votes Remaining', 'Votes16 Mcmulline',
       'Votes16 Hedgesj', 'Votes16 Kahnl', 'Votes16 La Rivag',
       'Votes16 Hoeflingt', 'Votes16 Kenistonc', 'Votes16 Smithm',
       'Votes16 Atwoodf', 'Votes16 Kennedya', 'Votes16 Kopitkek',
       'Votes16 Kotlikoffl', 'Votes16 Lyttleb', 'Votes16 Maldonadoj',
       'Votes16 Maturenm', 'Votes16 Scottr', 'Votes16 Silvar',
       'Votes16 Soltysike', 'Votes16 Vacekd', 'Votes16 Copelands',
       'Votes16 Jacobp', 'Votes16 Whitej', 'Votes16 Mooreheadm',
       'Votes16 None Of These Candidates', 'Votes16 Duncanr',
       'Votes16 Skewesp', 'Votes16 Giordanir'])

In [16]:
elecciones2.head()

,State,ST,Fips,County,Precincts,Votes,Republicans 2016,Democrats 2016,Green 2016,Libertarians 2016,Less Than High School Diploma,At Least High School Diploma,At Least Bachelors's Degree,Graduate Degree,School Enrollment,Median Earnings 2010,White (Not Latino) Population,African American Population,Native American Population,Asian American Population,Other Race or Races,Latino Population,Children Under 6 Living in Poverty,Adults 65 and Older Living in Poverty,Total Population,Preschool.Enrollment.Ratio.enrolled.ages.3.and.4,Poverty.Rate.below.federal.poverty.threshold,Gini.Coefficient,Child.Poverty.living.in.families.below.the.poverty.line,Management.professional.and.related.occupations,Service.occupations,Sales.and.office.occupations,Farming.fishing.and.forestry.occupations,Construction.extraction.maintenance.and.repair.occupations,Production.transportation.and.material.moving.occupations,White,Black,Hispanic,Asian,Amerindian,Other,White Asian,Sire Homogeneity,Median Age,lon,lat,Poor.physical.health.days,Poor.mental.health.days,Low.birthweight,Teen.births,Children.in.single.parent.households,Adult.smoking,Adult.obesity,Diabetes,Sexually.transmitted.infections,HIV.prevalence.rate,Uninsured,Unemployment,Violent.crime,Homicide.rate,Injury.deaths,Infant.mortality,Geo Shape,Name 16,reporting,Votes16 Trumpd,Votes16 Clintonh,Votes16 Johnsong,total16,Votos 16 otros
1,Georgia,GA,13089,"DeKalb County, Georgia",192,295871,16.06477147135069,80.82272341662413,NaN,3.112505112025173,12.1,87.9,38.7,15.1,76.15,31275.05492,30.05,53.6,0.2,4.6,1.65,9.95,25.75,10.75,712476,56.9,15.75,0.47200000000000003,24.0,41.3,15.1,25.45,0.05,8.15,9.85,30.05,53.6,9.95,4.6,0.2,1.65,34.65,0.38988875,33.9,-84.2263431,33.7706611,2.5,3.0,0.1,46.4,0.456,0.108,0.255,0.09,722.0,1080.4,0.23900000000000002,0.093,623.17,13.97,51.7,8.5,"{""type"": ""Polygon"", ""coordinates"": [[[-84.3484...",DeKalb,192,47531,239131,9209,295871.0,0.0
2,Texas,TX,48487,"Wilbarger County, Texas",7,4101,77.20068276030236,19.6781272860278,0.316995854669592,2.8041940990002443,26.4,73.6,15.9,5.9,69.25,22438.66508,63.3,8.9,0.7000000000000001,0.4,1.1,25.55,52.95,11.6,13630,32.4,24.3,0.454,37.05,26.25,25.5,18.1,2.35,12.95,14.75,63.3,8.9,25.55,0.4,0.7000000000000001,1.1,63.7,0.47407625000000003,36.6,-99.2424397,34.0849199,NaN,NaN,0.08,74.1,0.39,NaN,0.292,0.109,335.7,140.0,0.246,0.046,411.6,NaN,70.4,NaN,"{""type"": ""Polygon"", ""coordinates"": [[[-99.4752...",Wilbarger,7,3166,807,115,4101.0,0.0
3,Virginia,VA,51111,"Lunenburg County, Virginia",12,5577,57.48610363994979,39.913932221624535,0.448269679038909,1.5779092702169621,28.5,71.5,10.0,3.0,72.35,24528.98095,59.3,35.7,0.15,0.1,1.45,3.35,32.35,16.95,12911,35.4,18.55,0.392,27.35,21.15,19.65,26.75,1.9,10.75,19.85,59.3,35.7,3.35,0.1,0.15,1.45,59.4,0.48043375,43.0,-78.242313,36.9457567,4.2,3.1,0.08,49.6,0.377,0.214,0.338,0.134,582.6,239.0,0.191,0.084,182.27,NaN,92.5,NaN,"{""type"": ""Polygon"", ""coordinates"": [[[-78.3973...",Lunenburg,12,3206,2226,88,5577.0,0.0
4,Georgia,GA,13297,"Walton County, Georgia",21,40425,76.91527520098948,20.479901051329623,NaN,2.60482374768089,19.6,80.4,17.3,5.3,73.75,29155.95143,78.9,15.45,0.30000000000000004,1.1,1.0,3.1,22.25,8.85,83005,44.2,12.15,0.40700000000000003,17.7,26.8,14.8,29.15,0.4,14.05,14.75,78.9,15.45,3.1,1.1,0.30000000000000004,1.0,80.0,0.64758225,37.0,-83.7342152,33.7826485,3.0,2.9,0.08,45.0,0.248,0.164,0.28300000000000003,0.114,372.4,102.2,0.203,0.085,231.26,4.35,59.7,7.8,"{""type"": ""Polygon"", ""coordinates"": [[[-83.9156...",Walton,21,31093,8279,1053,40425.0,0.0
5,North Carolina,NC,37011,"Avery County, North Carolina",19,8060,77.24565756823822,20.719602977667495,NaN,2.034739454094293,18.7,81.3,20.3,6.0,76.25,19670.20603,89.85,3.3,0.30000000000000004,0.4,2.1,4.1,22.7,10.0,17850,52.9,15.6,0.529,20.85,29.7,22.15,21.35,1.75,12.25,12.75,89.85,3.3,4.1,0.4,0.30000000000000004,2.1,90.25,0.81053825,41.7,-81.920363,36.0728941,4.3,2.7,0.08,25.9,0.24,0.245,0.289,0.105,56.9,57.5,0.255,0.112,126.53,NaN,80.2,NaN,"{""ty

In [17]:
df = pd.DataFrame(elecciones2,
        columns=['State', 'ST', 'Fips', 'County', 'Precincts', 'Votes','Republicans 2016', 'Democrats 2016', 'Green 2016', 'Libertarians 2016','Geo Shape'])

In [18]:
df

,State,ST,Fips,County,Precincts,Votes,Republicans 2016,Democrats 2016,Green 2016,Libertarians 2016,Geo Shape
1,Georgia,GA,13089,"DeKalb County, Georgia",192,295871,16.06477147135069,80.82272341662413,NaN,3.112505112025173,"{""type"": ""Polygon"", ""coordinates"": [[[-84.3484..."
2,Texas,TX,48487,"Wilbarger County, Texas",7,4101,77.20068276030236,19.6781272860278,0.316995854669592,2.8041940990002443,"{""type"": ""Polygon"", ""coordinates"": [[[-99.4752..."
3,Virginia,VA,51111,"Lunenburg County, Virginia",12,5577,57.48610363994979,39.913932221624535,0.448269679038909,1.5779092702169621,"{""type"": ""Polygon"", ""coordinates"": [[[-78.3973..."
4,Georgia,GA,13297,"Walton County, Georgia",21,40425,76.91527520098948,20.479901051329623,NaN,2.60482374768089,"{""type"": ""Polygon"", ""coordinates"": [[[-83.9156..."
5,North Carolina,NC,37011,"Avery County, North Carolina",19,8060,77.24565756823822,20.719602977667495,NaN,2.034739454094293,"{""type"": ""Polygon"", ""coordinates"": [[[-82.0164..."
...,...,...,...,...,...,...,...,...,...,...,...
3139,Tennessee,TN,47131,"Obion County, Tennessee",12,12223,77.9268591998691,19.839646567945675,0.29452671193651303,1.578990427881862,"{""type"": ""Polygon"", ""coordinates"": [[[-89.2563..."
3140,Michigan,MI,26099,"Macomb County, Michigan",337,418773,53.630248368447816,42.08437506716049,0.927949032053164,3.07087610710337,"{""type"": ""Polygon"", ""coordinates"": [[[-82.7292..."
3141,West Virginia,WV,54007,"Braxton County, West Virginia",18,5063,69.44499308710252,25.97274343274738,0.9283033774442031,3.100928303377444,"{""type"": ""Polygon"", ""coordinates"": [[[-80.9333..."
3142,North Carolina,NC,37121,"Mitchell County, North Carolina",9,7948,78.32159033719175,19.94212380473075,NaN,1.736285858077503,"{""type"": ""Polygon"", ""coordinates"": [[[-82.2451..."


In [ ]:
pd.DataFrame = df.groupby(['State'])

In [ ]:
maps1 = gpd.read_file('shp/gadm36_BRA_1.shp')